# Economic Data

## Import libaries

In [2]:
import datetime
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import altair as alt
import os
import imgkit

## Set up functions and styles

In [208]:
#function that calculates the % change in the 
def per_change(x):
    return (x[-1]/x[0] - 1)

#set party color
def party_color(df):
    if df.Party == 'Democrat':
        return ['background-color: lightsteelblue']*len(df)
    else:
        return ['background-color: indianred']*len(df)


def remove_underscore(df):
    df.columns = df.columns.str.replace('_', ' ')
    return df
    
    
#table base style
df_base_style=({'background-color': 'black','color': 'black'})
#table header style
df_TH_style=   [{'selector': 'th', 'props': [('background-color', 'slategray'), ('color', 'white')]}]

#Set up html path
htmlpath=f'{os.getcwd()}\\html\\'

## Set query range from 1945 to 2020

In [4]:
start = datetime.datetime(1945, 1, 1)
end = datetime.datetime(2020, 1, 1)
date_range=pd.date_range(start=start, end=end)

In [5]:
'''Real Gross Domestic Product (GDPC1)
Gross Domestic Product (GDPA) (needed for GDP data prior to 1947)
Gross Domestic Product (GDP)
Gross domestic product (implicit price deflator) (A191RD3A086NBEA) (needed for GDP data prior to 1947)
Gross Domestic Product: Implicit Price Deflator (GDPDEF) --> use


Federal Debt: Total Public Debt (GFDEBTN)
Gross Federal Debt (FYGFD) --> annual date required prior to 1967
Federal Debt: Total Public Debt as Percent of Gross Domestic Product (GFDEGDQ188S)
Federal government budget surplus or deficit (-) (MTSDS133FMS)

Dates of U.S. recessions as inferred by GDP-based recession indicator (USRECM)
Federal Surplus or Deficit [-] (FYFSD)
Market Value of Gross Federal Debt (MVGFD027MNFRBDAL)
NBER based Recession Indicators for the United States from the Peak through the Trough (USRECM) -- replaces USRECM


'''

'Real Gross Domestic Product (GDPC1)\nGross Domestic Product (GDPA) (needed for GDP data prior to 1947)\nGross Domestic Product (GDP)\nGross domestic product (implicit price deflator) (A191RD3A086NBEA) (needed for GDP data prior to 1947)\nGross Domestic Product: Implicit Price Deflator (GDPDEF) --> use\n\n\nFederal Debt: Total Public Debt (GFDEBTN)\nGross Federal Debt (FYGFD) --> annual date required prior to 1967\nFederal Debt: Total Public Debt as Percent of Gross Domestic Product (GFDEGDQ188S)\nFederal government budget surplus or deficit (-) (MTSDS133FMS)\n\nDates of U.S. recessions as inferred by GDP-based recession indicator (USRECM)\nFederal Surplus or Deficit [-] (FYFSD)\nMarket Value of Gross Federal Debt (MVGFD027MNFRBDAL)\nNBER based Recession Indicators for the United States from the Peak through the Trough (USRECM) -- replaces USRECM\n\n\n'

## Query Federal Reserve Data

In [6]:
gdp = web.DataReader(['UNRATE','GDP','GFDEBTN','GDPDEF','USRECM','FYFSD'], 'fred', start, end)
federal_debt=web.DataReader(['FYGFD','GDPA','A191RD3A086NBEA'], 'fred', 1944, end) #need to go back further for federal debt for interpolation

### Correct historical public debt and GDP
- Historical public debt should be as of the end of the year, but this isn't the case. The quarterly debt is only available from FRED prior to 1966 so we need this data to backfill it. As such we will modify the datapull so the date is for the first day of the following year, then interpolate the quarterly values and resample assuming it increases the same amount per quarter as a proxy. The same is required for GDP data prior to 1947.

In [7]:
fdebt_annual=[date_index.strftime('01/%d/%Y') for date_index in list(federal_debt.index)]
federal_debt.set_index(pd.to_datetime(fdebt_annual),inplace=True)
federal_debt.head(2)


,FYGFD,GDPA,A191RD3A086NBEA
1944-01-01,NaN,224.447,9.544
1944-01-30,204.1,NaN,NaN


In [8]:
federal_debt_adj=federal_debt.resample('QS').sum()
federal_debt_adj=federal_debt_adj.replace(0, np.nan)
federal_debt_adj.interpolate(inplace=True)
federal_debt_adj.head()

,FYGFD,GDPA,A191RD3A086NBEA
1944-01-01,204.1,224.447,9.54400
1944-04-01,218.1,225.337,9.60575
1944-07-01,232.1,226.227,9.66750
1944-10-01,246.1,227.117,9.72925
1945-01-01,260.1,228.007,9.79100


In [9]:
gdp.head()

,UNRATE,GDP,GFDEBTN,GDPDEF,USRECM,FYFSD
DATE,,,,,,
1945-01-01,NaN,NaN,NaN,NaN,0.0,NaN
1945-02-01,NaN,NaN,NaN,NaN,1.0,NaN
1945-03-01,NaN,NaN,NaN,NaN,1.0,NaN
1945-04-01,NaN,NaN,NaN,NaN,1.0,NaN
1945-05-01,NaN,NaN,NaN,NaN,1.0,NaN


## Load datafile with presidents and party affiliation

In [10]:
presidents=pd.read_excel(f'{os.getcwd()}//data/Presidents.xlsx')

## Process presidents table
1. Convert the inaugaration date to Datetime.
2. Index table on date. This will be used to join to the Federal Reserve data.
3. Remove unecessary columns.
4. Forward fill presidents table in monthly format.

In [11]:
presidents['Year first inaugurated'] = pd.to_datetime(presidents['Term_Start'])
presidents.set_index(pd.DatetimeIndex(presidents['Term_Start']),inplace=True)
presidents.drop(['Vice-President','Year first inaugurated'],inplace=True,axis=1)
presidents=presidents.asfreq('MS',how='start',method='ffill')

presidents=presidents.reindex(index=date_range)
presidents=presidents.ffill()

In [12]:
presidents=presidents.loc[:'2020-01-01']
presidents.tail()

,President,Party,Term_Start,Days in Office
Term_Start,,,,
2019-09-01,Donald Trump,Republican,2017-01-01,1109.0
2019-10-01,Donald Trump,Republican,2017-01-01,1109.0
2019-11-01,Donald Trump,Republican,2017-01-01,1109.0
2019-12-01,Donald Trump,Republican,2017-01-01,1109.0
2020-01-01,Donald Trump,Republican,2017-01-01,1109.0


## Merge with Federal Reserve data
- Join on date index and confirm data after 1945 was joined properly.

In [13]:
gdp_pres = gdp.join(presidents,how='left')
gdp_pres=gdp_pres.drop(['UNRATE','GDP','GFDEBTN','GDPDEF','USRECM','FYFSD'],axis=1) 
gdp_pres.loc['1945-01-01':]

,President,Party,Term_Start,Days in Office
DATE,,,,
1945-01-01,Franklin Roosevelt,Democrat,1933-03-04,4422.0
1945-02-01,Franklin Roosevelt,Democrat,1933-03-04,4422.0
1945-03-01,Franklin Roosevelt,Democrat,1933-03-04,4422.0
1945-04-01,Franklin Roosevelt,Democrat,1933-03-04,4422.0
1945-05-01,Harry Truman,Democrat,1945-04-12,2840.0
...,...,...,...,...
2019-09-01,Donald Trump,Republican,2017-01-01,1109.0
2019-09-30,NaN,NaN,NaT,NaN
2019-10-01,Donald Trump,Republican,2017-01-01,1109.0


## Create GCP_chart data
### Resample GDP data into quarterly average for charting

In [14]:
gdp_chart=gdp.resample('QS').mean()
gdp_chart.tail()

,UNRATE,GDP,GFDEBTN,GDPDEF,USRECM,FYFSD
DATE,,,,,,
2018-10-01,3.800000,20897.804,21974096.0,111.256,0.0,NaN
2019-01-01,3.866667,21098.827,22027880.0,111.473,0.0,NaN
2019-04-01,3.633333,21340.267,22023283.0,112.188,0.0,NaN
2019-07-01,3.633333,21542.540,22719402.0,112.664,0.0,-984388.0
2019-10-01,3.533333,NaN,NaN,NaN,0.0,NaN


### Add in annual debt data

In [15]:
gdp_chart = gdp_chart.join(federal_debt_adj,how='left')

### Create calculated chart columns
1. Derive public debt
2. Divide nominal GDP / GDP deflator to get real quarterly GDP.
3. Rename columns to make them easier to reference.
4. Use real quarterly GDP to get QoQ change in GDP.
5. Remove excess columns

In [16]:
gdp_chart['Total_Public_Debt']=np.where(gdp_chart['GFDEBTN'].isna(),gdp_chart['FYGFD'],gdp_chart['GFDEBTN']/1000) #create_public_debt column. Need to convert the first column into millions for calculations
gdp_chart['GDP']=np.where(gdp_chart['GDP'].isna(),gdp_chart['GDPA'],gdp_chart.GDP ) #fill in missing nominal GPD data
gdp_chart['real_gdp']=np.where(gdp_chart['GDPDEF'].isna(),gdp_chart['GDPA'] /(gdp_chart.A191RD3A086NBEA)*100, gdp_chart.GDP / (gdp_chart.GDPDEF)*100 ) #add in mising GDP data and use deflator to get real GDP
gdp_chart['QoQ_real_GDP']=(gdp_chart.real_gdp / gdp_chart.real_gdp.shift(1)-1)

gdp_chart['debt_gdp_per']= np.where(gdp_chart.GFDEBTN.isna(),gdp_chart.FYGFD/ gdp_chart.GDP, gdp_chart.GFDEBTN / gdp_chart.GDP / 1000) #derive debt / GDP. Note for FYGFY is in billions and does not require an adjustment
gdp_chart['FYFSD']= gdp_chart.FYFSD / 1000  #scale federal deficit (annual) from millions to billions

#gdp_chart['Budget_Surplus_Deficit_GDP']=(gdp_chart['FYFSD'] / gdp_chart['Total_Public_Debt']) # budget surplus or deficit / total debt --> look to remove

gdp_chart['unemployment_rate']= gdp_chart.UNRATE / 100 #rescale unemployment rate

gdp_chart.rename(columns={'USRECM':'Recession','FYFSD':'Federal_Surplus_or_Deficit'},inplace=True)

In [352]:
#gdp_chart.head()
gdp_chart.tail()

,UNRATE,GDP,GFDEBTN,GDPDEF,Recession,Federal_Surplus_or_Deficit,FYGFD,GDPA,A191RD3A086NBEA,Total_Public_Debt,real_gdp,QoQ_real_GDP,debt_gdp_per,unemployment_rate,President,Party,Term_Start,Days_in_Office
DATE,,,,,,,,,,,,,,,,,,
2018-10-01,3.800000,20897.804,21974096.0,111.256,0.0,NaN,NaN,NaN,NaN,21974.096,18783.529877,0.002708,1.051503,0.038000,Donald Trump,Republican,2017-01-01,1109.0
2019-01-01,3.866667,21098.827,22027880.0,111.473,0.0,NaN,NaN,NaN,NaN,22027.880,18927.298090,0.007654,1.044033,0.038667,Donald Trump,Republican,2017-01-01,1109.0
2019-04-01,3.633333,21340.267,22023283.0,112.188,0.0,NaN,NaN,NaN,NaN,22023.283,19021.880237,0.004997,1.032006,0.036333,Donald Trump,Republican,2017-01-01,1109.0
2019-07-01,3.633333,21542.540,22719402.0,112.664,0.0,-984.388,NaN,NaN,NaN,22719.402,19121.050202,0.005213,1.054630,0.036333,Donald Trump,Republican,2017-01-01,1109.0
2019-10-01,3.533333,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.035333,Donald Trump,Republican,2017-01-01,1109.0


## Merge GDP chart data and presidents table

In [18]:
gdp_chart = gdp_chart.join(gdp_pres,how='left')

In [19]:
gdp_chart.rename(columns={'Days in Office':'Days_in_Office'},inplace=True)

In [20]:
gdp_chart
gdp_chart=gdp_chart.loc['1945-01-01':,]

### GDP Volatility including recessions

In [224]:
econ_party_summ=gdp_chart.groupby('Party').agg(
    Quarterly_GDP_Avg_Growth = pd.NamedAgg(column='QoQ_real_GDP', aggfunc=np.mean),
    Quarterly_GDP_Vol = pd.NamedAgg(column='QoQ_real_GDP', aggfunc=np.std),
    Avg_Unemployment_Rate = pd.NamedAgg(column='unemployment_rate', aggfunc=np.mean)
).sort_values(by='Quarterly_GDP_Avg_Growth',ascending=False)

#Remove underscores
econ_party_summ=(remove_underscore(econ_party_summ))

econ_party_summ_styled=econ_party_summ.reset_index().style.set_properties(**df_base_style).apply(party_color, axis=1).set_table_styles(df_TH_style).hide_index().applymap(lambda x: 'color: transparent' if pd.isnull(x) or x==0 else '').format({'Quarterly GDP Avg Growth': '{:,.2%}'.format,'Quarterly GDP Vol': '{:,.2%}'.format,'Avg Unemployment Rate': '{:,.2%}'.format})

In [225]:
econ_party_summ_styled

Party,Quarterly GDP Avg Growth,Quarterly GDP Vol,Avg Unemployment Rate
Democrat,0.78%,1.12%,5.68%
Republican,0.65%,0.94%,5.78%


In [226]:
econ_pres_summ=gdp_chart.groupby(['President','Party']).agg(
    Quarterly_GDP_Vol=pd.NamedAgg(column='QoQ_real_GDP', aggfunc=np.std),
    GDP_Start=pd.NamedAgg(column='real_gdp', aggfunc='first'),
    GDP_End=pd.NamedAgg(column='real_gdp', aggfunc='last'),
    GDP_Chg=pd.NamedAgg(column='real_gdp', aggfunc=per_change),
    Avg_Unemployment_Rate=pd.NamedAgg(column='unemployment_rate', aggfunc=np.mean),
    Debt_Start=pd.NamedAgg(column='Total_Public_Debt', aggfunc='first'),
    Debt_End=pd.NamedAgg(column='Total_Public_Debt', aggfunc='last'),
    Debt_Chg=pd.NamedAgg(column='Total_Public_Debt', aggfunc=per_change),
    Days_in_Office=pd.NamedAgg(column='Days_in_Office', aggfunc='mean')
).sort_values(by='GDP_Chg',ascending=False)

econ_pres_summ=(remove_underscore(econ_pres_summ))

In [227]:
econ_pres_summ_styled=econ_pres_summ.reset_index().style.set_properties(**df_base_style).apply(party_color, axis=1).set_table_styles(df_TH_style).hide_index().applymap(lambda x: 'color: transparent' if pd.isnull(x) or x==0 else '').format({
    'Quarterly GDP Vol': '{:,.2%}'.format, 'GDP Start': '${:,.1f}B'.format, 'GDP End': '${:,.1f}B'.format, 'GDP Chg': '{:,.1%}'.format,
    'Debt Start': '${:,.0f}B'.format, 'Debt End': '${:,.0f}B'.format, 'Debt Chg': '{:,.1%}'.format,'Avg Unemployment Rate': '{:,.2%}'.format})


In [223]:
econ_pres_summ_styled

President,Party,Quarterly GDP Vol,GDP Start,GDP End,GDP Chg,Avg Unemployment Rate,Debt Start,Debt End,Debt Chg,Days in Office
Bill Clinton,Democrat,0.47%,"$9,850.9B","$13,260.5B",34.6%,5.20%,"$4,231B","$5,662B",33.8%,2922
Ronald Reagan,Republican,0.90%,"$6,947.0B","$9,009.8B",29.7%,7.54%,$965B,"$2,684B",178.3%,2922
Lyndon Johnson,Democrat,0.77%,"$3,851.3B","$4,844.8B",25.8%,4.17%,$316B,$358B,13.3%,1886
Harry Truman,Democrat,1.81%,"$2,185.4B","$2,648.6B",21.2%,4.26%,$266B,$264B,-0.5%,2840
Dwight Eisenhower,Republican,1.30%,"$2,697.9B","$3,232.0B",19.8%,4.89%,$266B,$292B,9.8%,2922
Barack Obama,Democrat,0.51%,"$15,156.0B","$17,824.2B",17.6%,7.45%,"$11,127B","$19,977B",79.5%,2922
George W. Bush,Republican,0.70%,"$13,222.7B","$15,328.1B",15.9%,5.27%,"$5,774B","$10,700B",85.3%,2922
John F. Kennedy,Democrat,0.60%,"$3,253.8B","$3,771.9B",15.9%,5.97%,$293B,$315B,7.5%,1036
Richard Nixon,Republican,1.08%,"$4,920.7B","$5,638.4B",14.2%,5.02%,$360B,$481B,29.4%,2027
Jimmy Carter,Democrat,1.28%,"$6,079.4B","$6,813.5B",12.1%,6.54%,$669B,$930B,39.0%,1461


## Setting up the first set of economic charts
1. Set up chart size/width and ranges for the yscales.
2. Create a top and bottom layer of charts. The layers consist of a line chart to show the trend, a circle/scatter chart used to mark the points (needed for interactive tooltips), and an area chart to indicate recessions. One looks at quarter-over-quarter changes in GDP and the other looks at unemployment.
3. The time interval for both charts is linked to the 'brush' selector, which is a chart placed in between the two charts. This allows the user to change the time intervals for the analysis.

### Economic Growth and Unemployment

In [376]:
source['mean_q']*100

0      0.714312
1      0.714312
2      0.714312
3      0.714312
4      0.714312
         ...   
295    0.714312
296    0.714312
297    0.714312
298    0.714312
299    0.714312
Name: mean_q, Length: 300, dtype: float64

In [377]:
# Set chart size/specs
chart_height=450
chart_width=1200

# Set range for y-axis on both charts
yrange_gdp= (min(gdp_chart.QoQ_real_GDP.dropna())*1.1, max(gdp_chart.QoQ_real_GDP.dropna()*1.1))

yrange_unemp= (min(gdp_chart.unemployment_rate.dropna())*1.1, max(gdp_chart.unemployment_rate.dropna()*1.1))

#parameter for opacity. Used to shade recessions.
recession_op=0.2

# Selector to change time interval for the charts
brush = alt.selection(type='interval', encodings=['x'],empty='none')
selection1 = alt.selection_single()


source=gdp_chart.reset_index()
source['mean_q']=np.mean(source.QoQ_real_GDP)


#Base for the top chart
upper1= (alt.Chart(source).encode(
    x=alt.X('DATE:T',scale=alt.Scale(domain=brush),axis=alt.Axis(labelAngle=-45,title=None)),
    y=alt.Y('QoQ_real_GDP:Q',impute=alt.ImputeParams(value=None),
            axis=alt.Axis(format='%'),
          scale=alt.Scale(domain=yrange_gdp),title='Quarter-over-Quarter Change in Real GDP'),)
    .properties(width=chart_width,height=chart_height,title= 'Post-WW2 US Quarterly Change in GDP, Mean Quarterly Growth = ~0.714% (~2.89% annualized )')
        )

#Top line chart and circle pointers for the tooltips
upper1_line=upper1.mark_line(color='grey')
upper1_circle=upper1.mark_circle().encode(
    tooltip=[alt.Tooltip('DATE',format='%b %Y',title='Date'), 'President', alt.Tooltip('QoQ_real_GDP',format='.2%',title="Quarter-over-Quarter Change in Real GDP"),alt.Tooltip('real_gdp',format='$,.2f',title='Real GDP (USD Billions)')],
    color=alt.Color('Party',scale=alt.Scale(domain=['Democrat','Republican'], range=['darkblue','maroon']),legend=alt.Legend(title="Political Party",orient='bottom'))
).add_selection(selection1)

upper1_mean = alt.Chart(source).mark_rule(color='black').encode(y=alt.Y('mean_q'))



#Set up chart to shade recessions
upper1_recession= upper1.mark_area(yOffset=-5,y2Offset=1000).encode(
    y=alt.Y('QoQ_real_GDP:Q',impute=alt.ImputeParams(value=None),
    scale=alt.Scale(domain = yrange_gdp)),
    color=alt.Color('Recession:O',scale=alt.Scale(domain=['0','1'], range=['transparent','black']),legend=None),opacity=alt.value(recession_op)
    )


#Base for the bottom chart
lower1= (alt.Chart(source).encode(
    x=alt.X('DATE:T',scale=alt.Scale(domain=brush),axis=alt.Axis(labelAngle=-45,title=None)),
    y=alt.Y('unemployment_rate:Q',impute=alt.ImputeParams(value=None),
            axis=alt.Axis(format='%'),
          scale=alt.Scale(domain = yrange_unemp),title="Average Quarterly Unemployment Rate")
    )
    .properties(width=chart_width,height=chart_height, title= 'Post WW2-US Unemployment Rate (Quarterly)')
        )

#Bottom line chart and circle pointers for the tooltips
lower1_line=lower1.mark_line(color='grey')
lower1_circle=lower1.mark_circle().encode(
    tooltip=[alt.Tooltip('DATE',format='%b %Y',title='Date'), 'President', alt.Tooltip('unemployment_rate',format='.2%',title="Unemployment Rate"),alt.Tooltip('real_gdp',format='$,.2f',title='Real GDP (USD Billions)')],
    color=alt.Color('Party',scale=alt.Scale(domain=['Democrat','Republican'], range=['darkblue','maroon']),legend=None)).add_selection(selection1)

#Set up recessions for lower chart
lower1_recession= lower1.mark_area().encode(
    y=alt.Y('unemployment_rate:Q',impute=alt.ImputeParams(value=None),
    scale=alt.Scale(domain = yrange_unemp)),
    color=alt.Color('Recession:N',scale=alt.Scale(domain=['0','1'], range=['transparent','black']),legend=None),opacity=alt.value(recession_op))

#Set up chart used to select time intervals
date_range = (alt.Chart(source)
         .mark_rule(size=5,color='black')
         .encode(
    alt.X('DATE:T',scale=alt.Scale(),title='Highlight selection here to filter by Date Range. Shaded area represent recessions. Source: St. Louis Federal Reserve.'))
         .properties(width=chart_width,height=10)
         .add_selection(brush)   
)

econ1_chart = alt.layer(upper1_line, upper1_circle, upper1_mean, upper1_recession).resolve_scale(color='independent') & date_range & alt.layer(lower1_line, lower1_circle, lower1_recession).resolve_scale(color='independent')

# Save the chart as HTML file
econ1_chart.save(f'{htmlpath}econ1_chart.html')

#Display the chart
econ1_chart



alt.VConcatChart(...)

### Debt and Deficit Charts

In [ ]:
#gdp_chart.head()
gdp_chart.tail()

In [348]:
# Set chart size/specs

# Set range for y-axis on both charts
yrange_gdp= (min(gdp_chart['Total_Public_Debt'].dropna())*1.1, max(gdp_chart['Total_Public_Debt'].dropna()*1.1))

yrange_unemp= (min(gdp_chart.debt_gdp_per.dropna())*1.1, max(gdp_chart.debt_gdp_per.dropna()*1.1))

#parameter for opacity. Used to shade recessions.
recession_op=0.2

# Selector to change time interval for the charts
brush = alt.selection(type='interval', encodings=['x'],empty='none')
selection1 = alt.selection_single()

#Base for the top chart
upper1= (alt.Chart(source).encode(
    x=alt.X('DATE:T',scale=alt.Scale(domain=brush),axis=alt.Axis(labelAngle=-45,title=None)),
    y=alt.Y('Total_Public_Debt:Q',impute=alt.ImputeParams(value=None),
            axis=alt.Axis(format='$,.0f'),
          scale=alt.Scale(domain=yrange_gdp),title='Total (Nominal) Public Debt (Billions)'),)
    .properties(width=chart_width,height=chart_height,title= 'Post-WW2 Total Public Debt')
        )

#Top line chart and circle pointers for the tooltips
upper1_line=upper1.mark_line(color='grey')
upper1_circle=upper1.mark_circle().encode(
    tooltip=[alt.Tooltip('DATE',format='%b %Y',title='Date'), 'President', alt.Tooltip('Total_Public_Debt',format='$,.0f',title="Total (Nominal) Public Debt (Billions)"),alt.Tooltip('real_gdp',format='$,.2f',title='Real GDP (USD Billions)')],
    color=alt.Color('Party',scale=alt.Scale(domain=['Democrat','Republican'], range=['darkblue','maroon']),legend=alt.Legend(title="Political Party",orient='bottom'))
).add_selection(selection1)


#bar chart of federal deficit
budget = (alt.Chart(source)
         .mark_bar(color='grey')
         .encode(
    alt.X('DATE:T',scale=alt.Scale(domain=brush),title='Year',axis=None),
    y=alt.Y('Federal_Surplus_or_Deficit:Q',impute=alt.ImputeParams(value=None),title='Federal Surplus/Deficit (Millions)',axis=alt.Axis(format='$,.0f',gridOpacity =0,orient='right'))
         ).properties(width=chart_width,height=200)
         )

budget_text = budget.mark_text(
    align='left',
    baseline='top',
    dx=0,
    dy=-1000,  # Nudges text to right so it doesn't appear on top of the bar
    angle=270
).encode(
    text=alt.Text('Federal_Surplus_or_Deficit:Q',format='$,.0f')
)



#Set up chart to shade recessions
upper1_recession = upper1.mark_area(yOffset=-5,y2Offset=1000).encode(
    y=alt.Y('Total_Public_Debt:Q',impute=alt.ImputeParams(value=None),
    scale=alt.Scale(domain = yrange_gdp)),
    color=alt.Color('Recession:O',scale=alt.Scale(domain=['0','1'], range=['transparent','black']),legend=None),opacity=alt.value(recession_op)
    )


#Base for the bottom chart
lower1= (alt.Chart(source).encode(
    x=alt.X('DATE:T',scale=alt.Scale(domain=brush),axis=alt.Axis(labelAngle=-45,title=None)),
    y=alt.Y('debt_gdp_per:Q',impute=alt.ImputeParams(value=None),
            axis=alt.Axis(format='%'),
          scale=alt.Scale(domain = yrange_unemp),title='Debt to GDP %')
    )
    .properties(width=chart_width,height=chart_height, title= 'Post-WW2 Debt-to-GDP %')
        )

#Bottom line chart and circle pointers for the tooltips
lower1_line=lower1.mark_line(color='grey')
lower1_circle=lower1.mark_circle().encode(
    tooltip=[alt.Tooltip('DATE',format='%b %Y',title='Date'), 'President', alt.Tooltip('debt_gdp_per',title='Debt to GDP %',format='.2%'),alt.Tooltip('real_gdp',format='$,.2f',title='Real GDP (USD Billions)')],
    color=alt.Color('Party',scale=alt.Scale(domain=['Democrat','Republican'], range=['darkblue','maroon']),legend=None)).add_selection(selection1)



#Set up recessions for lower chart
lower1_recession= lower1.mark_area().encode(
    y=alt.Y('debt_gdp_per:Q',impute=alt.ImputeParams(value=None),
    scale=alt.Scale(domain = yrange_unemp)),
    color=alt.Color('Recession:N',scale=alt.Scale(domain=['0','1'], range=['transparent','black']),legend=None),opacity=alt.value(recession_op)
)

#Set up chart used to select time intervals
date_range = (alt.Chart(source)
         .mark_rule(size=5,color='black')
         .encode(
    alt.X('DATE:T',scale=alt.Scale(),title='Highlight selection here to filter by Date Range. Shaded area represent recessions. Source: St. Louis Federal Reserve.'))
         .properties(width=chart_width,height=10)
         .add_selection(brush)   
)

budget.configure_view(
    stroke="transparent")

budget_text.configure_axis(
    labelFontSize=50
).configure_text(
    fontSize=20
)

econ2_chart = alt.layer(upper1_line,upper1_circle, upper1_recession).resolve_scale(color='independent')  & date_range & alt.layer(lower1_line, lower1_circle, lower1_recession).resolve_scale(color='independent')

# Save the chart as HTML file
econ2_chart.save(f'{htmlpath}econ2_chart.html')

#Display the chart
econ2_chart 


alt.VConcatChart(...)

# Equity Data

## Import libaries and pull data

In [228]:
import quandl
SandP = quandl.get("MULTPL/SP500_REAL_PRICE_MONTH")

## Merge equity return data to presidents and recession data

In [229]:
df_equity = SandP.loc['1945-01-01':,:].join(presidents,how='left')
df_equity = df_equity.join(gdp['USRECM'],how='left')

## Rename Headers

In [230]:
df_equity.rename(index={'Date':'DATE'},columns={'Value':'SP500_Real_Monthly_Price','USRECM':'Recession_Indicator'},inplace=True)
df_equity.index.names=['DATE']

## Derive Monthly Market Returns

In [231]:
df_equity['MoM_SP500_Return']=(df_equity['SP500_Real_Monthly_Price'] / df_equity['SP500_Real_Monthly_Price'].shift(1)) - 1

In [232]:
df_equity.Recession_Indicator.ffill(0,inplace=True)

In [233]:
df_equity.style.format({'SP500_Real_Monthly_Price': '{:,.2f}'.format, 'MoM_SP500_Return': '{:,.2%}'.format})
df_equity.head()

,SP500_Real_Monthly_Price,President,Party,Term_Start,Days in Office,Recession_Indicator,MoM_SP500_Return
DATE,,,,,,,
1945-01-01,13.49,Franklin Roosevelt,Democrat,1933-03-04,4422.0,0.0,NaN
1945-02-01,13.94,Franklin Roosevelt,Democrat,1933-03-04,4422.0,1.0,0.033358
1945-03-01,13.93,Franklin Roosevelt,Democrat,1933-03-04,4422.0,1.0,-0.000717
1945-04-01,14.28,Franklin Roosevelt,Democrat,1933-03-04,4422.0,1.0,0.025126
1945-05-01,14.82,Harry Truman,Democrat,1945-04-12,2840.0,1.0,0.037815


## Get summary data
1. Data aggregated by Party.
2. Data aggregated by President.

### Equity Volatility including recessions

In [409]:
df_equity_summ= df_equity.groupby('Party').agg(
    Monthly_Vol=pd.NamedAgg(column='MoM_SP500_Return', aggfunc=np.std)
).sort_values(by='Monthly_Vol',ascending=False)

df_equity_summ=(remove_underscore(df_equity_summ))

df_equity_summ.reset_index().style.set_properties(**df_base_style).apply(party_color, axis=1).set_table_styles(df_TH_style).hide_index().applymap(lambda x: 'color: transparent' if pd.isnull(x) or x==0 else '').format({'Monthly Vol': '{:,.2%}'.format})

Party,Monthly Vol
Republican,3.66%
Democrat,3.17%


In [239]:
df_equity_pres_summ=df_equity.groupby(['President','Party']).agg(
    Monthly_Vol=pd.NamedAgg(column='MoM_SP500_Return', aggfunc=np.std),
    SP500_Start=pd.NamedAgg(column='SP500_Real_Monthly_Price', aggfunc='first'),
    SP500_End=pd.NamedAgg(column='SP500_Real_Monthly_Price', aggfunc='last'),
    SP500_Chg=pd.NamedAgg(column='SP500_Real_Monthly_Price', aggfunc=per_change)
).sort_values(by='SP500_Chg',ascending=False)

df_equity_pres_summ=(remove_underscore(df_equity_pres_summ))

In [240]:
df_equity_pres_summ.reset_index().style.set_properties(**df_base_style).apply(party_color, axis=1).set_table_styles(df_TH_style).hide_index().applymap(lambda x: 'color: transparent' if pd.isnull(x) or x==0 else '').format({'Monthly Vol': '{:,.2%}'.format, 'SP500 Start': '{:,.1f}'.format, 'SP500 End': '{:,.1f}'.format, 'SP500 Chg': '{:,.1%}'.format}).applymap(lambda x: 'color: transparent' if pd.isnull(x) else '')

President,Party,Monthly Vol,SP500 Start,SP500 End,SP500 Chg
Bill Clinton,Democrat,2.94%,435.2,"1,330.9",205.8%
Barack Obama,Democrat,3.27%,865.6,"2,246.6",159.6%
Dwight Eisenhower,Republican,2.93%,26.2,56.8,117.0%
Ronald Reagan,Republican,3.84%,133.0,276.5,107.9%
Harry Truman,Democrat,3.50%,14.8,26.0,75.7%
Gerald Ford,Republican,4.65%,68.1,104.7,53.7%
George H. W. Bush,Republican,3.03%,285.4,435.6,52.6%
Lyndon Johnson,Democrat,2.55%,74.2,106.5,43.6%
Donald Trump,Republican,2.56%,"2,275.1","3,159.8",38.9%
Jimmy Carter,Democrat,3.24%,103.8,133.5,28.6%


## Build Equity Charts

In [397]:

0.006698*100

0.6698

In [400]:
# Set chart size/specs
chart_height=300
chart_width=1000

# Set range for y-axis on both charts
yrange_price= (min(df_equity.SP500_Real_Monthly_Price.dropna())*1.05, max(df_equity.SP500_Real_Monthly_Price.dropna()*1.05))

yrange_return= (min(df_equity.MoM_SP500_Return.dropna())*1.05, max(df_equity.MoM_SP500_Return.dropna()*1.05))

#parameter for opacity. Used to shade recessions.
recession_op=0.2

# Selector to change time interval for the charts
brush = alt.selection(type='interval', encodings=['x'],empty='none')
selection1 = alt.selection_single()

source1=df_equity.reset_index()
source1['mean_return']=df_equity.MoM_SP500_Return.mean()

#Base for the top chart
upper1= (alt.Chart(source1).encode(
    x=alt.X('DATE:T',scale=alt.Scale(domain=brush),axis=alt.Axis(labelAngle=-45,title=None)),
    y=alt.Y('SP500_Real_Monthly_Price:Q',impute=alt.ImputeParams(value=None),
            axis=alt.Axis(),
          scale=alt.Scale(domain=yrange_price),title='S&P 500 Real Price by Month'),)
    .properties(width=chart_width,height=chart_height,title= 'S&P 500, Post-WWII Monthly Price Data')
        )

#Top line chart and circle pointers for the tooltips
upper1_line=upper1.mark_line(color='grey')
upper1_circle=upper1.mark_circle(size=15).encode(
    tooltip=[alt.Tooltip('DATE',format='%b %Y'), 'President', 'SP500_Real_Monthly_Price'],
    color=alt.Color('Party',scale=alt.Scale(domain=['Democrat','Republican'], range=['darkblue','maroon']),legend=alt.Legend(title="Political Party",orient='bottom'))
).add_selection(selection1)

#Set up chart to shade recessions
upper1_recession= upper1.mark_area(yOffset=0,y2Offset=0).encode(
    y=alt.Y('SP500_Real_Monthly_Price:Q',impute=alt.ImputeParams(value=None),
    scale=alt.Scale(domain = yrange_price)),
    color=alt.Color('Recession_Indicator:O',scale=alt.Scale(domain=['0','1'], range=['transparent','black']),legend=None),opacity=alt.value(recession_op)
    )

lower1_mean = alt.Chart(source1).mark_rule(color='black').encode(y=alt.Y('mean_return'))

#Base for the bottom chart
lower1= (alt.Chart(source1).encode(
    x=alt.X('DATE:T',scale=alt.Scale(domain=brush),axis=alt.Axis(labelAngle=-45,title=None)),
    y=alt.Y('MoM_SP500_Return:Q',impute=alt.ImputeParams(value=None),
          scale=alt.Scale(domain = yrange_return),axis=alt.Axis(format='%',title="S&P 500 Month-over-Month % Change"))
    )
    .properties(width=chart_width,height=chart_height, title= 'S&P 500 Month-over-Month Change, Post-WWII, Mean Monthly Return = ~0.6698% (~2.7% annualized)' )
        )
#Bottom line chart and circle pointers for the tooltips
lower1_line=lower1.mark_line(color='grey')
lower1_circle=lower1.mark_circle().encode(
    tooltip=[alt.Tooltip('DATE',format='%b %Y'), alt.Tooltip('President'), alt.Tooltip('MoM_SP500_Return',format='.2%')],
    color=alt.Color('Party',scale=alt.Scale(domain=['Democrat','Republican'], range=['darkblue','maroon']),legend=None)).add_selection(selection1)

#Set up recessions for lower chart
lower1_recession= lower1.mark_area(yOffset=0,y2Offset=1000).encode(
    y=alt.Y('MoM_SP500_Return:Q',impute=alt.ImputeParams(value=None),
    scale=alt.Scale(domain = yrange_return)),
    color=alt.Color('Recession_Indicator:O',scale=alt.Scale(domain=['0','1'], range=['transparent','black']),legend=None),opacity=alt.value(recession_op))

#Set up chart used to select time intervals
date_range = (alt.Chart(source1)
         .mark_rule(size=5,color='black')
         .encode(
    alt.X('DATE:T',scale=alt.Scale(),title='Highlight selection here to filter by Date Range. Shaded area represent recessions. Recession data available after 1967. Source: St. Louis Federal Reserve and Quandl.'))
         .properties(width=chart_width,height=10)
         .add_selection(brush)   
)

df_equity_chart = alt.layer(upper1_line, upper1_circle, upper1_recession).resolve_scale(color='independent') & date_range & alt.layer(lower1_line, lower1_circle, lower1_mean, lower1_recession).resolve_scale(color='independent')

# Save the chart as HTML file
df_equity_chart.save(f'{htmlpath}df_equity_chart.html')

#Display the chart
df_equity_chart



alt.VConcatChart(...)